In [1]:
import openfermionpyscf as ofpyscf
import openfermion as of
import cirq
import openfermioncirq as ofc
import numpy as np
import pyscf

In [8]:
from openfermion.transforms import get_fermion_operator, jordan_wigner, bravyi_kitaev, get_interaction_operator, bravyi_kitaev_fast
geometry = [["H", [0, 0, 0]],
            ["H", [0, 0, 0.7414]]]
basis = "sto-3g"
multiplicity = 1
charge = 0
hamiltonian = ofpyscf.generate_molecular_hamiltonian(geometry, basis, multiplicity, charge)
hamiltonian_bk = bravyi_kitaev(get_fermion_operator(hamiltonian))
fermion= get_fermion_operator(hamiltonian)
interaction= get_interaction_operator(fermion)
bkf=bravyi_kitaev_fast(interaction)
print(bkf)

(-0.09886396933545713+0j) [] +
(0.045322202052874+0j) [X0 X3] +
(0.045322202052874+0j) [Y0 Z1 Z2 Y3] +
(0.045322202052874+0j) [Y0 Y3] +
(-0.045322202052874+0j) [Z0 X1 X2 Z3] +
(-0.045322202052874+0j) [Z0 Y1 Y2 Z3] +
(0.17119774903432944+0j) [Z0 Z1] +
(0.241089644106036+0j) [Z0 Z1 Z2 Z3] +
(0.17119774903432947+0j) [Z0 Z2] +
(0.331734048211784+0j) [Z0 Z3] +
(-0.045322202052874+0j) [Y1 Y2] +
(0.34297063344496626+0j) [Z1 Z2] +
(-0.22278593040418498+0j) [Z1 Z3] +
(-0.22278593040418498+0j) [Z2 Z3]


In [3]:
from openfermion.ops import QubitOperator
from openfermion.utils import pauli_exp_to_qasm
for line in pauli_exp_to_qasm([QubitOperator('X2 Y0',0.0)]):
    print(line)

Rx 1.5707963267948966 0
H 2
CNOT 0 2
Rz 0.0 2
CNOT 0 2
Rx -1.5707963267948966 0
H 2


In [17]:
#Membuat Sirkuit Ansatz /Fungsi Gelombang Coba 
class Ansatz(ofc.VariationalAnsatz):
    def params(self):      
        return [cirq.Symbol('theta_0')]
    def operations(self, qubits):    
        q0, q1,q2,q3= qubits
        yield cirq.XPowGate(exponent=1.5707963267948966).on(q0)
        yield cirq.H(q2)
        yield cirq.CNOT(q0,q2)
        yield cirq.ZPowGate(exponent=cirq.Symbol('theta_0')).on(q2)
        yield cirq.CNOT(q0,q2)
        yield cirq.XPowGate(exponent=-1.5707963267948966).on(q0)
        yield cirq.H(q2)
        
        
        
        
        
        
        
    def _generate_qubits(set):
        return cirq.LineQubit.range(4)

In [18]:
FungsiCoba=Ansatz()
print(FungsiCoba.circuit)

0: ───X^(-3/7)───@───────────────@───X^(3/7)───
                 │               │
2: ───H──────────X───Z^theta_0───X───H─────────


In [19]:
#mendeklarasikan Hamiltonan yang akan ditinjau
objektif = ofc.HamiltonianObjective(hamiltonian_bk)

#membuat keadaan referensi
q0,q1, _,_  = FungsiCoba.qubits
keadaan_refensi = cirq.Circuit.from_ops(cirq.X(q0))


study = ofc.VariationalStudy(
    name='VQE JW Hidrogen',
    ansatz=FungsiCoba,
    objective=objektif,
    preparation_circuit=keadaan_refensi)

In [20]:
print(study.circuit)

0: ───X───X^(-3/7)───@───────────────@───X^(3/7)───
                     │               │
2: ───────H──────────X───Z^theta_0───X───H─────────


In [21]:
initial_guess=FungsiCoba.default_initial_params()
print(initial_guess)

[0.]


In [22]:
#optimasi
algoritma = ofc.optimization.ScipyOptimizationAlgorithm(
    kwargs={'method': 'SLSQP'},
    options={'maxiter': 100,'disp': True})


optimization_params = ofc.optimization.OptimizationParams(
    algorithm=algoritma,
    initial_guess=FungsiCoba.default_initial_params())


result = study.optimize(optimization_params)
print('Nilai optimum: {}'.format(result.optimal_value))
print("parameter optimasinya:")
print(result.optimal_parameters)

Optimization terminated successfully.    (Exit mode 0)
            Current function value: -1.1360122619551216
            Iterations: 3
            Function evaluations: 10
            Gradient evaluations: 3
Nilai optimum: -1.1360122619551216
parameter optimasinya:
[-0.06923998]


In [10]:
#optimasi
algoritma = ofc.optimization.ScipyOptimizationAlgorithm(
    kwargs={'method': 'nelder-mead'},
    options={'maxiter': 100,'disp': True})


optimization_params = ofc.optimization.OptimizationParams(
    algorithm=algoritma,
    initial_guess=FungsiCoba.default_initial_params())


result = study.optimize(optimization_params)
print('Nilai optimum: {}'.format(result.optimal_value))
print("parameter optimasinya:")
print(result.optimal_parameters)

Optimization terminated successfully.
         Current function value: -1.116684
         Iterations: 3
         Function evaluations: 6
Nilai optimum: -1.1166843870853398
parameter optimasinya:
[0.]


In [66]:
#optimasi
algoritma = ofc.optimization.ScipyOptimizationAlgorithm(
    kwargs={'method': 'powell'},
    options={'maxiter': 100,'disp': True})


optimization_params = ofc.optimization.OptimizationParams(
    algorithm=algoritma,
    initial_guess=FungsiCoba.default_initial_params())


result = study.optimize(optimization_params)
print('Nilai optimum: {}'.format(result.optimal_value))
print("parameter optimasinya:")
print(result.optimal_parameters)

Optimization terminated successfully.
         Current function value: -1.136012
         Iterations: 2
         Function evaluations: 45
Nilai optimum: -1.1360122825409535
parameter optimasinya:
-0.06931136471332987


In [65]:
#optimasi
algoritma = ofc.optimization.ScipyOptimizationAlgorithm(
    kwargs={'method': 'L-BFGS-B'},
    options={'maxiter': 100,'disp': True})


optimization_params = ofc.optimization.OptimizationParams(
    algorithm=algoritma,
    initial_guess=FungsiCoba.default_initial_params())


result = study.optimize(optimization_params)
print('Nilai optimum: {}'.format(result.optimal_value))
print("parameter optimasinya:")
print(result.optimal_parameters)

Nilai optimum: -1.136012282540943
parameter optimasinya:
[-0.06931132]


In [12]:
algoritma = ofc.optimization.ScipyOptimizationAlgorithm(
    kwargs={'method': 'COBYLA'},
    uses_bounds=False)


optimization_params = ofc.optimization.OptimizationParams(
    algorithm=algoritma,
    initial_guess=FungsiCoba.default_initial_params())


result = study.optimize(optimization_params)
print('Nilai optimum: {}'.format(result.optimal_value))
print("parameter optimasinya:")
print(result.optimal_parameters)

Nilai optimum: -1.136012259559471
parameter optimasinya:
[-0.06923594]


In [19]:
import scipy
print(scipy.optimize.show_options(solver='minimize',method='COBYLA' for 'minimize',disp=True))

SyntaxError: invalid syntax (<ipython-input-19-113b0d40ffef>, line 2)